In [1]:
import pandas as pd
import altair as alt

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
df_siRNAs_path = '/Users/tobias/Library/CloudStorage/Dropbox/Science_02/Projects/High-Throughput_Screen/siRNA_library.xlsx'

df_siRNAs = pd.read_excel(df_siRNAs_path, sheet_name = "Plate_01_split", usecols = ["Well", "Name", "Cells"])
df_siRNAs.head()

,Well,Name,Cells
0,B2,LUC1_init,IM
1,B3,KIF11_orig,IM
2,B4,ANAPC11_new,IM
3,B5,APC_new,IM
4,B6,ASPM_init,IM


In [3]:
def merge_tables(table_path, RNAtable = df_siRNAs):
    list_of_sheets = ["24h", "48h", "72h"]
    dataframes = []
    for sheet in list_of_sheets:
        df = pd.read_excel(table_path, sheet_name = sheet)
        df["Well"] = df.Row + df.Column.astype(str)
        df["Timepoint"] = sheet
        df = df.merge(df_siRNAs)
        dataframes.append(df)

    final_df = pd.concat(dataframes)
    return final_df

In [4]:
df_analysis_path = '/Volumes/arxivBeta/_Tobias/Opera/20250204/20250211_Mitotic Index_10x_Analysis.xlsx'

df = merge_tables(df_analysis_path)

In [5]:
area_FOV = 1640961 # µm2
df["Population_density_cells_per_mm2"] = (df['Modified Nuclei - Number of Objects'] / area_FOV) * 1e6

In [6]:
df.columns

Index(['Display', 'Use for Z'', 'Row', 'Column', 'Plane', 'Timepoint',
       'Modified Nuclei - Number of Objects',
       'Modified Nuclei - Region Area [µm²] - Mean per Well',
       'Modified Nuclei - Region Area [µm²] - StdDev per Well',
       'Mitotic cells - Number of Objects', 'Mitotic Index (%)',
       'Number of Analyzed Fields', 'Height [µm]', 'Time [s]', 'Compound',
       'Concentration', 'Cell Type', 'Cell Count', 'Well', 'Name', 'Cells',
       'Population_density_cells_per_mm2'],
      dtype='object')

In [7]:
group_MI = df.groupby(["Cells", 'Timepoint'])["Mitotic Index (%)"].mean()
group_MI

Cells  Timepoint
CM     24h          1.300547
       48h          0.656873
       72h          0.180381
IM     24h          1.368520
       48h          1.617400
       72h          1.533767
Name: Mitotic Index (%), dtype: float64

In [8]:
colourscheme = "viridis"

In [9]:
def Binned_Boxplot(
    data, x, y, x_title, y_title, color, column, 
    Boxsize = 10,  
    Bin_Boxplot_width = 100, 
    Bin_Boxplot_height = 180):
    # Boxplot for binned x data
    BINNED_BOX = alt.Chart(
        data = data, 
        width = Bin_Boxplot_width, 
        height = Bin_Boxplot_height
    ).mark_boxplot(
        size = Boxsize, 
        extent = "min-max"
    ).encode(
    alt.X(x, title = None, axis = alt.Axis(grid = False)),
    alt.Y(y,  title = y_title, axis = alt.Axis(grid = False)),
    color = alt.Color(
        color, scale = alt.Scale(scheme = colourscheme)
    ),
    column = column
    ).configure_facet(
            spacing = 15
    ).configure_axis(
                grid = True, ticks = True, labelPadding = 5, offset = 10
    ).configure_header(
            labelOrient = 'bottom', title = None
    ).configure_view(
            stroke = 'transparent', 
            strokeWidth = 0.5
    )
    return BINNED_BOX

In [10]:
def Binned_Boxplot2(
    data, x, y, x_title, y_title, color, column, 
    Boxsize = 7,  
    Bin_Boxplot_width = 300, 
    Bin_Boxplot_height = 180):
    # Boxplot for binned x data
    BINNED_BOX = alt.Chart(
        data = data, 
        width = Bin_Boxplot_width, 
        height = Bin_Boxplot_height
    ).mark_boxplot(
        size = Boxsize, 
        extent = "min-max"
    ).encode(
    alt.X(x, title = None, axis = alt.Axis(grid = False)),
    alt.Y(y,  title = y_title, axis = alt.Axis(grid = False)),
    color = alt.Color(
        color, scale = alt.Scale(scheme = colourscheme)
    ),
    column = column
    ).configure_facet(
            spacing = 15
    ).configure_axis(
                grid = True, ticks = True, labelPadding = 5, offset = 10
    ).configure_header(
            labelOrient = 'bottom', title = None
    ).configure_view(
            stroke = 'transparent', 
            strokeWidth = 0.5
    )

    
    return BINNED_BOX

In [11]:
def Binned_Circles(
    data, x, y, x_title, y_title, color, column, 
    Boxsize = 100,  
    Bin_Boxplot_width = 400, 
    Bin_Boxplot_height = 180):
    # Boxplot for binned x data
    BINNED_BOX = alt.Chart(
        data = data, 
        width = Bin_Boxplot_width, 
        height = Bin_Boxplot_height
    ).mark_circle(
        size = Boxsize,
    ).encode(
    alt.X(x, title = None, axis = alt.Axis(grid = False)),
    alt.Y("mean(" + y + ")",  title = y_title, axis = alt.Axis(grid = False)),
    color = alt.Color(
        color, scale = alt.Scale(scheme = colourscheme)
    ),
    column = column
    ).configure_facet(
            spacing = 15
    ).configure_axis(
                grid = True, ticks = True, labelPadding = 5, offset = 15
    ).configure_header(
            labelOrient = 'bottom', title = None
    ).configure_view(
            stroke = 'transparent', 
            strokeWidth = 0.5
    )    
    
    ERROR_Circles = alt.Chart(
            data = data
    ).mark_errorbar(extent = "stdev").encode(
        alt.X(x, title = x_title), 
        alt.Y("mean(" + y + ")", title = y_title), #scale = alt.Scale(domain = [y_min, y_max])),
        color = alt.Color(
            color, scale = alt.Scale(scheme = colourscheme)
    ),
    column = column
    ).configure_facet(
            spacing = 15
    ).configure_axis(
                grid = True, ticks = True, labelPadding = 5, offset = 10
    ).configure_header(
            labelOrient = 'bottom', title = None
    ).configure_view(
            stroke = 'transparent', 
            strokeWidth = 0.5
    )

    BINNED_Circle = BINNED_BOX
    return BINNED_Circle

In [12]:
def Scatterbin(dataframe, x, y, color, x_title, y_title, binextent, binstep, x_min, x_max, y_min, y_max,
            Circlesize = 10, 
            CircleOpacity = 0.4,  
            Scatter_width = 200, 
            Scatter_height = 200
               ):
    # Standard scatter plot 
    SCATTER = alt.Chart(
        data = dataframe, 
        width = Scatter_width, 
        height = Scatter_height
    ).mark_circle(
        opacity = CircleOpacity,
        size = Circlesize
    ).encode(
        alt.X(x, title = x_title, scale = alt.Scale(domain = [x_min, x_max])),
        alt.Y(y, title = y_title, scale = alt.Scale(domain = [y_min, y_max])),
        color = alt.Color(
            color, scale = alt.Scale(scheme = colourscheme)#, legend = None#, scale = None
        ) 
    )
    
    SCATTER_bin = alt.Chart(
        data = dataframe, 
        width = Scatter_width, 
        height = Scatter_height
    ).mark_circle(
        opacity = 1,
        size = 150
    ).encode(
        alt.X(x, title = x_title, bin = alt.Bin(extent = binextent, step = binstep)), #scale = alt.Scale(domain = [x_min, x_max])),
        alt.Y("median(" + y + ")", title = y_title, bin = False), #scale = alt.Scale(domain = [y_min, y_max])),
        color = alt.Color(
            color, scale = alt.Scale(scheme = colourscheme)#, legend = None#, scale = None
        ) 
    )
    
    ERROR_Scatterbin = alt.Chart(
            data = dataframe
    ).mark_errorbar(extent = "iqr").encode(
        alt.X(x, title = x_title, bin = alt.Bin(extent = binextent, step = binstep)), #scale = alt.Scale(domain = [x_min, x_max])),
        alt.Y("median(" + y + ")", title = y_title, bin = False), #scale = alt.Scale(domain = [y_min, y_max])),
        color = alt.Color(
            color, scale = alt.Scale(scheme = colourscheme)#, legend = None#, scale = None
        ) 
    ) 

    
    SCATTERBIN = SCATTER + ERROR_Scatterbin + SCATTER_bin 
    return SCATTERBIN

In [13]:
BINBOX_MitoticIndex_Time = Binned_Boxplot(
    data = df,
    x = 'Timepoint',
    y = 'Mitotic Index (%)',
    x_title = '',
    y_title = 'Mitotic Index (%)',
    color = 'Cells',
    column = 'Cells'
)
BINBOX_MitoticIndex_Time

alt.Chart(...)

In [14]:
SCATTERBIN_MI_Pop = Scatterbin(
    dataframe = df, 
    x = "Modified Nuclei - Number of Objects", 
    y = "Mitotic Index (%)", 
    color = "Cells", 
    x_title = "Number of nuclei in FOV", 
    y_title = "Mitotic Index (%)", 
    binextent = [0, 7000], 
    binstep = 1000,
    x_min = 2500,
    x_max = 7500,
    y_min = 0,
    y_max = 2.4
)
SCATTERBIN_MI_Pop

alt.LayerChart(...)

In [15]:
SCATTERBIN_MI_Pop2 = Scatterbin(
    dataframe = df, 
    x = "Population_density_cells_per_mm2", 
    y = "Mitotic Index (%)", 
    color = "Cells", 
    x_title = "Cells per area (1/mm2)", 
    y_title = "Mitotic Index (%)", 
    binextent = [0, 5000], 
    binstep = 500,
    x_min = 1500,
    x_max = 5000,
    y_min = 0,
    y_max = 2.4
)
SCATTERBIN_MI_Pop2

alt.LayerChart(...)

In [16]:
BINBOX_MitoticIndex_Time_Targets = Binned_Circles(
    data = df,
    column = 'Cells',
    y = 'Mitotic Index (%)',
    x_title = '',
    y_title = 'Mitotic Index (%)',
    color = 'Cells',
    x = 'Name'
)
BINBOX_MitoticIndex_Time_Targets

alt.Chart(...)